# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
city_weather = pd.read_csv("../Output/weather_check.csv")
city_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Belyy Yar,53.60,91.39,44.60,76,26,13.42,RU,1602680885
1,1,Tuktoyaktuk,69.45,-133.04,20.73,95,82,10.25,CA,1602680341
2,2,Ushuaia,-54.80,-68.30,42.80,60,40,25.28,AR,1602680803
3,3,Albany,42.60,-73.97,51.01,80,0,5.64,US,1602680333
4,4,Ribeira Grande,38.52,-28.70,75.20,64,20,9.17,PT,1602680336


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
max_humidity = humidity.max()

heat_map_figure = gmaps.figure()
heat_map_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = max_humidity, 
                                    point_radius = 3)

heat_map_figure.add_layer(heat_map_layer)

#heat_map_figure

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
narrow_down_cities_df = city_weather.loc[(city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & 
                                         (city_weather["Cloudiness"] == 0), :]

narrow_down_cities_df = narrow_down_cities_df.dropna(how = "any")

del narrow_down_cities_df["Unnamed: 0"]

narrow_down_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Cape Town,-33.93,18.42,79.00,31,0,8.05,ZA,1602680119
31,Hermanus,-34.42,19.23,77.00,46,0,5.99,ZA,1602680047
52,Grand Gaube,-20.01,57.66,79.00,70,0,5.99,MU,1602680487
59,Port Hedland,-20.32,118.57,77.00,78,0,5.82,AU,1602680889
128,Zaraza,9.35,-65.32,79.48,86,0,4.09,VE,1602680895


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
hotels_df = narrow_down_cities_df
hotels = []

for i in range (len(hotels_df)):
    latitude = hotels_df["Lat"]
    longitude = hotels_df["Lng"]
    
    params = {
        "location" : f"{latitude},{longitude}", 
        "radius" : 5000, 
        "types" : "hotel", 
        "key" : "g_key"
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params)
    response_json = response.json()
    
    try:
        hotels.append(response_json["results"][0]["name"])
    
    except (KeyError, IndexError): 
        hotels.append("N/A")
    
    time.sleep(1.1)

In [71]:
hotel_df["Hotel Name"] = hotels
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Cape Town,-33.93,18.42,79.00,31,0,8.05,ZA,1602680119,N/A
31,Hermanus,-34.42,19.23,77.00,46,0,5.99,ZA,1602680047,N/A
52,Grand Gaube,-20.01,57.66,79.00,70,0,5.99,MU,1602680487,N/A
59,Port Hedland,-20.32,118.57,77.00,78,0,5.82,AU,1602680889,N/A
128,Zaraza,9.35,-65.32,79.48,86,0,4.09,VE,1602680895,N/A


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
